______
<img style="float: right;" src="/files/tutorial/images/headerlogo.png">
## Google Earth Engine Tutorial
### Automatic data load
_____

In a previous notebook data were load to a GEE asset using the Javascript interface. In this section, a script will be used to upload several images contained in a folder. 

In [5]:
__author__ = 'yang'
__maintainer__ = 'Raul Zurita-Milla' , 'Rosa Aguilar'
__email__ = "r.zurita-milla@utwente.nl", "rosamaguilar@gmail.com"

# -------------------------------------------------------------------------------
# Name:        gee_upload_asset_v3loop_v6
# Purpose:     Load Images to assets in GEE
# 
# -------------------------------------------------------------------------------

#----------IMPORTS & INITS
import glob
import requests
from urllib import parse
import os
from os import path
import ee
from requests_toolbelt.multipart import encoder
import logging
import sys
import ast
from bs4 import BeautifulSoup

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
ee.Initialize()


#--------------------Parameters-------------------------------------
username =      # your gmail account
password =        # your password



# # ASSET folder or collection name and ASSET name:
# your asset id
asset_dir = 
# your local directory
img_dir =  
# suffix for the file description that will be created for each image. This file will contain the assetid of the image in GEE 
name = "_desc.txt"   # suffix



#---------- Define URLs---------------------------------------------------
google_accounts_url = 'https://accounts.google.com'
authentication_url = 'https://accounts.google.com/ServiceLoginAuth'
appspot_url = 'https://ee-api.appspot.com/assets/upload/geturl?'

def google_authenticate(username, password):
    session = requests.session()

    login_html = session.get(google_accounts_url)
    soup_login = BeautifulSoup(login_html.content, 'html.parser').find('form').find_all('input')
    payload = {}
    for u in soup_login:
        if u.has_attr('value'):
            payload[u['name']] = u['value']

    payload['Email'] = username
    payload['Passwd'] = password

    auto = login_html.headers.get('X-Auto-Login')
    follow_up = parse.unquote(parse.unquote(auto)).split('continue=')[-1]
    galx = login_html.cookies['GALX']

    payload['continue'] = follow_up
    payload['GALX'] = galx

    session.post(authentication_url, data=payload)
    return session


def get_uploadUrl(session):
    '''
    get the GEE asset upload url
    '''
    _ = session.get('https://ee-api.appspot.com/assets/upload/geturl?')
    r = session.get('https://ee-api.appspot.com/assets/upload/geturl?')
    if r.text.startswith('\n<!DOCTYPE html>'):
        logging.error('Incorrect credentials. Probably. If you are sure the credentials are OK, refresh the authentication token. '
                      'If it did not work report a problem. They might have changed something in the Matrix.')
        sys.exit(1)
    d = ast.literal_eval(r.text)

    return d['url']


def upload_file(session, file_path, asset_name, use_multipart, properties=None, nodata=None):

    with open(file_path, 'rb') as f:
        upload_url = get_uploadUrl(session)

        if use_multipart:
            form = encoder.MultipartEncoder({
                "documents": (file_path, f, "application/octet-stream"),
                "composite": "NONE",
            })
            headers = {"Prefer": "respond-async", "Content-Type": form.content_type}
            resp = session.post(upload_url, headers=headers, data=form)
        else:
            files = {'file': f}
            resp = session.post(upload_url, files=files)

        gsid = resp.json()[0]
        asset_data = {"id": asset_name,
                      "tilesets": [
                          {"sources": [
                              {"primaryPath": gsid,
                               "additionalPaths": []
                              }
                          ]}
                      ],
                      "bands": [],
                      "properties": properties,
                      "missingData": {"value": 255}
                      }
    return asset_data



#----------Main Section
username = ''
password = ''

google = google_authenticate(username, password)
print ('Authenticated')

# # ASSET folder or collection name and ASSET name:
asset_dir = 'users/rosamaguilar/tutorial/'
img_dir = 'E:/itc-stars/tutorial/uploadimages/'
name = "_Multi.txt"   # suffix
os.chdir(img_dir)
files = glob.glob('*.tif')
# LOOP all TIF images in a given folder
for n in range(0,len(files)):
    im = files[n]
    asset_name = asset_dir + files[n][:-4]
    print ("asset", n, asset_name)
    #Local file path
    img = img_dir + im    #+'/'+ files[n]
    print (img)
    textFileName = img_dir + im[:-4] + name
    if os.path.isfile (textFileName):
        print (textFileName)
        print ('file already in GEE')

    else:
        print ('uploading file')
        asset_request = upload_file(google, img, asset_name, False)
        taskid = ee.data.newTaskId(1)[0]
        print ('ingesting...', taskid)
        ret = ee.data.startIngestion(taskid, asset_request)
        print ('ingesting...', ret)
        asset_request = 0

        with open(textFileName, "w") as text_file:
            text_file.write(str.format(asset_name))


NameError: name 'username' is not defined